In [2]:
# Importing necessary libraries
import oså

import numpy as np=
import tensorflow as tf
from tensorflow.keras.datasets import mnist
from bokeh.plotting import figure, output_file, save, show
from bokeh.models import Range1d, LinearAxis, FactorRange, ColumnDataSource, LabelSet, LinearColorMapper
from bokeh.io import output_notebook
from matplotlib import gridspec, colors
from bokeh.palettes import Greys256

from bokeh.models import ColumnDataSource, LabelSet
from bokeh.transform import transform

import matplotlib.pyplot as plt
import numpy as np

2023-07-14 09:15:30.156842: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-07-14 09:15:30.309028: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-07-14 09:15:30.310969: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-07-14 09:15:32.494823: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
def plot_sample_digits(x_train, y_train):
    # Convert one-hot encoded y_train back to label format
    y_train_labels = np.argmax(y_train.numpy(), axis=1)
    
    # Convert x_train to numpy
    x_train_np = x_train.numpy()

    # Prepare an example of each digit
    examples = {}
    for i in range(10):
        examples[i] = x_train_np[np.argwhere(y_train_labels == i)[0][0]].reshape(28, 28)
        
    # Prepare the figure
    fig, axes = plt.subplots(2, 5, figsize=(10,4))

    for i in range(10):
        ax = axes[i//5, i%5]
        ax.imshow(examples[i], cmap='gray', aspect='auto')
        ax.axis('off')  # hide the axes ticks
    
    output_file_path = "generated/mnist_examples.png"
        
    # Check/Create model directory
    os.makedirs(output_file_path, exist_ok=True)å

    plt.subplots_adjust(wspace=0.1, hspace=0.1)
    plt.savefig(output_file_path)

In [ ]:
# Plot loss and accuracy over epochs
def plot_metrics(loss_per_epoch, accuracy_per_epoch):
    # Create a new plot with a range for the loss [left]
    p = figure(x_axis_label='Epochs', y_axis_label='Loss')
    
    output_notebook()
    
    # Set range for loss axis
    loss_max = max(loss_per_epoch) + 0.05 * max(loss_per_epoch)  # for example, add 5% as a padding to the max value
    p.y_range = Range1d(0, loss_max)  # adjust according to your loss data
    
    # Plot loss
    p.line(np.arange(len(loss_per_epoch)) + 1, loss_per_epoch, legend_label="Loss", line_color="red", line_width = 3)
    
    # Create a new range for the accuracy [right]
    p.extra_y_ranges = {"AccuracyRange": Range1d(start=0, end=100)} # Update the range
    p.add_layout(LinearAxis(y_range_name="AccuracyRange", axis_label="Accuracy (%)"), 'right') # Update the label

    # Scale accuracy values to percentage and plot
    accuracy_per_epoch_percentage = [acc * 100 for acc in accuracy_per_epoch] # Scale accuracy
    p.line(np.arange(len(accuracy_per_epoch_percentage)) + 1, accuracy_per_epoch_percentage, legend_label="Accuracy", line_color="blue", y_range_name="AccuracyRange", line_width = 3) # Plot scaled accuracy
    
    # Remove the grid
    p.xgrid.grid_line_color = None
    p.ygrid.grid_line_color = None
    
    output_file_path = "generated/metrics_plot.html"
    
    # Check/Create model directory
    os.makedirs(output_file_path, exist_ok=True)
    
    # Save the plot
    output_file(output_file_path)
    show(p)

In [26]:
def plot_model_output(model, output_digit, instance, x_data, y_data):
    # Convert one-hot encoded y_train back to label format
    y_labels = np.argmax(y_data, axis=1)
    
    output_notebook()
    
    # Find the first instance of '0'
    zero_index = np.argwhere(y_labels == output_digit)[instance][0]
    zero_input = tf.reshape(x_data[zero_index], (1, -1)) # Reshape it to match model input shape

    # Calculate model output
    output = model(zero_input).numpy()[0]

    # Prepare data for bar chart
    digits = [str(i) for i in range(10)][::-1]
    output_values = output.tolist()[::-1]
    
    # Format output values to have 2 decimal places
    formatted_output_values = [f"{val:.2E}" for val in output_values]

    # Create a new figure with a white background
    p = figure(y_range=FactorRange(factors=digits), 
               width=500, 
               title="Model Output for First Instance of Digit 0",
               y_axis_label='Digits',
               x_axis_label='Model Output (~Probability)',
               background_fill_color="white")

    # Create a ColumnDataSource with the data
    source = ColumnDataSource(data=dict(digits=digits, output_values=output_values, formatted_output_values=formatted_output_values))

    # Add a horizontal bar chart to the figure with black bars
    p.hbar(y='digits', right='output_values', height=0.5, color="black", source=source)

    # Create labels for the bars with color based on a condition
    color_mapper = LinearColorMapper(palette=['black', 'white'], low=min(output_values), high=max(output_values))
    labels = LabelSet(x='output_values', y='digits', text='formatted_output_values', level='annotation',
                      x_offset=5, y_offset=-10, text_align='left', text_color=transform('output_values', color_mapper), source=source)
    
    print(formatted_output_values)
    # Add labels to the plot
    p.add_layout(labels)

    p.xaxis.axis_label_text_font_size = "20pt"
    p.yaxis.axis_label_text_font_size = "20pt"
    p.xaxis.major_label_text_font_size = "15pt"
    p.yaxis.major_label_text_font_size = "15pt"

    # Remove the grid
    p.xgrid.grid_line_color = None
    p.ygrid.grid_line_color = None
    
    output_file_path = f"generated/output_plot_for_{output_digit}.html"
    
    # Check/Create model directory
    os.makedirs(output_file_path, exist_ok=True)

    # Save the plot
    output_file(output_file_path)
    show(p)

In [4]:
# Step 1: Load and prepare the MNIST dataset.
def load_data():
    # This data is already split into train and test datasets.
    (x_train, y_train), (x_test, y_test) = mnist.load_data()

    # Reshape and normalize the images to feed into the neural network.
    x_train, x_test = tf.cast(x_train.reshape(-1, 784)/255.0, tf.float32), \
    tf.cast(x_test.reshape(-1, 784)/255.0, tf.float32)

    # Convert labels to one-hot vectors. This is necessary as our output layer 
    # will have 10 neurons, one for each digit from 0 to 9.
    y_train, y_test = tf.one_hot(y_train, depth=10), tf.one_hot(y_test, depth=10)
    
    return x_train, y_train, x_test, y_test

In [22]:
# Load the MNIST dataset
train_images, train_labels, test_images, test_labels = load_data()

# Define the model
model = tf.keras.models.Sequential([
  # The first dense layer
  tf.keras.layers.Dense(128, activation='relu', input_shape=(784,)),
  # The output layer
  tf.keras.layers.Dense(10, activation='softmax')
])

# Compile the model
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Train the model
history = model.fit(train_images, train_labels, epochs=15)

# Evaluate the model on the test data
loss_per_epoch, accuracy_per_epoch = model.evaluate(test_images, test_labels)

Epoch 1/15
1875/1875 [==============================] - 6s 3ms/step - loss: 0.2641 - accuracy: 0.9249
Epoch 2/15
1875/1875 [==============================] - 6s 3ms/step - loss: 0.1171 - accuracy: 0.9651
Epoch 3/15
1875/1875 [==============================] - 5s 3ms/step - loss: 0.0796 - accuracy: 0.9754
Epoch 4/15
1875/1875 [==============================] - 6s 3ms/step - loss: 0.0593 - accuracy: 0.9821
Epoch 5/15
1875/1875 [==============================] - 5s 2ms/step - loss: 0.0455 - accuracy: 0.9857
Epoch 6/15
1875/1875 [==============================] - 6s 3ms/step - loss: 0.0368 - accuracy: 0.9888
Epoch 7/15
1875/1875 [==============================] - 6s 3ms/step - loss: 0.0280 - accuracy: 0.9919
Epoch 8/15
1875/1875 [==============================] - 6s 3ms/step - loss: 0.0223 - accuracy: 0.9931
Epoch 9/15
1875/1875 [==============================] - 6s 3ms/step - loss: 0.0184 - accuracy: 0.9942
Epoch 10/15
1875/1875 [==============================] - 6s 3ms/step - loss: 0.015

In [24]:
plot_metrics(history.history['loss'], history.history['accuracy'])

Loading BokehJS ...

In [30]:
plot_model_output(model, 5, 2, train_images, train_labels)

Loading BokehJS ...

['5.29E-07', '1.20E-07', '9.58E-11', '9.64E-07', '1.00E+00', '1.70E-10', '1.53E-07', '1.13E-13', '5.04E-10', '1.37E-08']


In [31]:
print(loss_per_epoch, accuracy_per_epoch)

0.08710840344429016 0.9800000190734863
